In [3]:
# import vaex
from pandarallel import pandarallel
pandarallel.initialize()
import pandas as pd
import os
from pathlib import Path
import sys
sys.path.append('./../..')
sys.path.append('./..')
from common_utils import utils

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
DATA_SAVE_DIR = 'Processed'
pathobj = Path(DATA_SAVE_DIR)
pathobj.mkdir(exist_ok=True, parents=True)


In [5]:
def get_IPs():
    tmp = pd.read_csv(
        'ugr16_ip_list.txt',index_col=None,delimiter=';'
    )
    labels = tmp['label']
    ips = tmp['IP']
    _dict = {}
    
    for i,j in zip(ips,labels):
        first3= '.'.join(i.split('.')[:-1])
        last4 = i.split('.')[-1]
        last4 = last4.strip('{')
        last4 = last4.strip('}')
        last4 = last4.split(',')
        for k in last4:
            _dict[first3 + '.' + k] = j
    return _dict
Known_IPs = get_IPs()


In [6]:
attacker_list = [k for k,v in Known_IPs.items() if v == 'attacker']
victim_list = [k for k,v in Known_IPs.items() if v == 'victim']

In [7]:
# ===========================
# Column names need to be injected
# ============================
cols ='te,td,sa,da,sp,dp,pr,flg,fwd,stos,pkt,byt,label'.split(',')
print(cols)

['te', 'td', 'sa', 'da', 'sp', 'dp', 'pr', 'flg', 'fwd', 'stos', 'pkt', 'byt', 'label']


In [8]:
DATA_FILE = './../Data_Raw/uniq/april.week3.csv.uniqblacklistremoved'

# ==================================================
# df_list = vaex.from_csv(
#     data_file,
#     chunk_size = 100000,
#     names = cols,
#     header = None
# )
# ===================================================

In [9]:
start_TS = '2016-04-12 00:00:00'
end_TS = '2016-04-12 23:59:59'
df_filtered_time = None

In [10]:
reader = pd.read_csv(DATA_FILE, index_col=None, low_memory=False, names =cols, header=None, chunksize=100000)

In [11]:
df_filtered_time = None

In [12]:
def validate(value,_list):
    if value in _list:
        return value
    else:
        return None

In [ ]:
for chunk in reader:
    df_pd = chunk
    
    df_pd['TS'] = pd.to_datetime(df_pd['te'])
    # ==============================
    # Check if in time period
    # ===============================
    tmp = df_pd[(df_pd['TS'] > start_TS) & (df_pd['TS'] < end_TS) ]
    
    if len(tmp)==0: 
        continue

    tmp['da'] = tmp['da'].parallel_apply(
        validate,
        args=(victim_list,)
    )
    
    tmp = tmp.dropna()
    
    if len(tmp)==0: 
        continue
    
    print(
        len(tmp),
        df_pd.head(5)
    )
    if len(tmp) > 0 :
        if df_filtered_time is None:
            df_filtered_time = tmp
        else:
            df_filtered_time = df_filtered_time.append(tmp,ignore_index=True)
    if len(df_filtered_time) > 0  and len(tmp) == 0 :
        break
        

In [ ]:
len(df_filtered_time)

In [ ]:
# =======================
# Save the data
# =======================
file_name = 'data_' + start_TS.split(' ')[0] + '.csv'
file_path = os.path.join(DATA_SAVE_DIR, file_name)
utils.save_csv(
    df_filtered_time,
    file_path
)
# ====
# Use utils.fetch_csv to read files
# ====




In [35]:
sshscan_file = './../../../../UGR16/april/week3/sshscan_flows_cut.csv'
df = pd.read_csv(data_file, header=None, names=cols)

In [36]:
df.head(10)

,te,td,sa,da,sp,dp,pr,flg,fwd,stos,pkt,byt,label
0,2016-04-10 12:09:17,0.292,42.219.156.231,71.63.255.244,36676,22,TCP,.A..SF,0,0,3,164,anomaly-sshscan
1,2016-04-10 12:09:17,0.312,42.219.156.231,71.63.255.243,57609,22,TCP,.A..SF,0,0,3,164,anomaly-sshscan
2,2016-04-10 12:09:17,0.288,42.219.156.231,71.63.255.242,50803,22,TCP,.A..SF,0,0,3,164,anomaly-sshscan
3,2016-04-10 12:09:17,0.316,42.219.156.231,71.63.255.240,41397,22,TCP,.A..SF,0,0,3,164,anomaly-sshscan
4,2016-04-10 12:09:17,0.288,42.219.156.231,71.63.255.250,32906,22,TCP,.A..SF,0,0,3,164,anomaly-sshscan
5,2016-04-10 12:09:17,0.000,42.219.156.231,71.63.254.62,40564,22,TCP,....S.,0,0,1,60,anomaly-sshscan
6,2016-04-10 12:09:17,0.000,42.219.156.231,71.63.254.51,54708,22,TCP,....S.,0,0,1,60,anomaly-sshscan
7,2016-04-10 12:09:17,0.000,42.219.156.231,71.63.254.49,40607,22,TCP,....S.,0,0,1,60,anomaly-sshscan
8,2016-04-10 12:09:17,0.000,42.219.156.231,71.63.254.46,57127,22,TCP,....S.,0,0,1,60,anomaly-sshscan
9,2016-04-10 12:09:17,0.000,42.219.156.231,71.63.254.37,54796,22,TCP,....S.,0,0,1,60,anomaly-sshscan
